In [1]:
#data cleanming i have done using R language and building model using Python
import numpy as np
import pandas as pd
import os
from sklearn.multiclass import OneVsOneClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing,cross_validation

C:\Users\Sai\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
os.chdir('G:\downloads')
df=pd.read_csv('intern1.csv')

In [3]:
df.head()

,Unnamed: 0,category,gender,sub_category,sub_sub_category
0,2876,kids & baby,baby & toddler,baby,baby gear
1,7883,clothing,mens,sweaters,None
2,4089,sports & outdoors,None,team shop,team clothing
3,8828,health & beauty,womens,beauty,makeup & fragrance
4,9401,clothing,mens,suits,suit separates


In [4]:
df=df[['category','gender','sub_category','sub_sub_category']]
df.head()

,category,gender,sub_category,sub_sub_category
0,kids & baby,baby & toddler,baby,baby gear
1,clothing,mens,sweaters,None
2,sports & outdoors,None,team shop,team clothing
3,health & beauty,womens,beauty,makeup & fragrance
4,clothing,mens,suits,suit separates


In [5]:
df.shape

(7000, 4)

In [6]:
df.category.value_counts()

sports & outdoors         2612
clothing                  1557
accessories                417
shoes                      367
kids & baby                237
home & pet                 196
health & beauty            102
electronics                 75
automotive                  31
toys & games                27
books, movies & music        5
flowers & food gifting       2
Name: category, dtype: int64

In [7]:
#converting  category to numeric
df['category_num']=df.category.map({'sports & outdoors':1,'clothing':2,'accessories':3,'shoes':4,'kids & baby':5,'home & pet':6,'health & beauty':7,'electronics':8,'automotive':9,'toys & games':10,'books, movies & music':11,'flowers & food gifting':12})
df.head()

,category,gender,sub_category,sub_sub_category,category_num
0,kids & baby,baby & toddler,baby,baby gear,5.0
1,clothing,mens,sweaters,None,2.0
2,sports & outdoors,None,team shop,team clothing,1.0
3,health & beauty,womens,beauty,makeup & fragrance,7.0
4,clothing,mens,suits,suit separates,2.0


In [8]:
df.gender.value_counts()

unisex            2836
womens            1551
None              1253
mens               659
baby & toddler     151
plus size           88
girls               78
boys                52
kids                32
baby girls          13
baby boys           11
Name: gender, dtype: int64

In [9]:
df['gender_num']=df.gender.map({'unisex':1,'womens':2,'None':3,'mens':4,'baby & toddler':5,'plus size':6,'girls':7,'boys':8,'kids':9,'baby girls':10,'baby boys':11})
x=df.sub_category.unique()
x.shape

(89,)

In [10]:
#here i am creating labels for sub_category
x
df['sub_category_num']=df.sub_category.map({k: v for v, k in enumerate(x,1)})

In [11]:
y=df.sub_sub_category.unique()

In [12]:
#here i am creating labels for sub_sub_category
y
df['sub_sub_category_num']=df.sub_sub_category.map({k: v for v, k in enumerate(y,1)})
df.head(10)

,category,gender,sub_category,sub_sub_category,category_num,gender_num,sub_category_num,sub_sub_category_num
0,kids & baby,baby & toddler,baby,baby gear,5.0,5.0,1,1
1,clothing,mens,sweaters,None,2.0,4.0,2,2
2,sports & outdoors,None,team shop,team clothing,1.0,3.0,3,3
3,health & beauty,womens,beauty,makeup & fragrance,7.0,2.0,4,4
4,clothing,mens,suits,suit separates,2.0,4.0,5,5
5,sports & outdoors,unisex,team shop,team clothing,1.0,1.0,3,3
6,clothing,womens,tops,None,2.0,2.0,6,2
7,sports & outdoors,None,team shop,None,1.0,3.0,3,2
8,sports & outdoors,None,team shop,team clothing,1.0,3.0,3,3
9,sports & outdoors,unisex,team shop,team clothing,1.0,1.0,3,3


In [13]:
#converting the above columns into float type
df['sub_category_num'] = df['sub_category_num'].astype('float64')
df['sub_sub_category_num'] = df['sub_sub_category_num'].astype('float64')
df.head(5)

,category,gender,sub_category,sub_sub_category,category_num,gender_num,sub_category_num,sub_sub_category_num
0,kids & baby,baby & toddler,baby,baby gear,5.0,5.0,1.0,1.0
1,clothing,mens,sweaters,None,2.0,4.0,2.0,2.0
2,sports & outdoors,None,team shop,team clothing,1.0,3.0,3.0,3.0
3,health & beauty,womens,beauty,makeup & fragrance,7.0,2.0,4.0,4.0
4,clothing,mens,suits,suit separates,2.0,4.0,5.0,5.0


In [14]:
df1=df[['category_num','gender_num','sub_category_num','sub_sub_category_num']]
df1.head(60)
df2=df1[~df1.isnull().any(1)]

In [15]:
#here iam seperating dependent and independent variables
b=np.array(df2.drop(['gender_num'],1)) 

In [16]:
#i used factorize because there are more than 1 features and converting into 2d array
c=pd.factorize(df2['gender_num'].values)[0].reshape(-1,1)

In [17]:
b_train, b_test, c_train, c_test=cross_validation.train_test_split(b,c,test_size=0.2,random_state=42)

In [18]:
#As this is multi class 
clf_tree = DecisionTreeClassifier(max_depth = 2).fit(b_train, c_train)
predictions = clf_tree.predict(b_test)

In [19]:
accuracy=clf_tree.score(b_test,c_test)

In [20]:
accuracy

0.60657193605683835

In [21]:
#confusion matrix
cm = confusion_matrix(c_test,predictions)

In [22]:
cm

array([[  0,   0,   0,  28,   8,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0, 139,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,  25, 107,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0, 287,   1,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,  79, 396,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   5,   3,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,  20,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,  12,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,  11,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   2,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   3,   0,   0,   0,   0,   0,   0,   0]], dtype=int64)

In [23]:
#test data 
df12=pd.read_csv('Book1.csv')

In [24]:
clf_tree.predict(df12)

array([3, 3, 4, 3, 3], dtype=int64)

In [25]:
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix

In [26]:
clf4 = LinearSVC(random_state=0)

In [27]:
clf4.fit(b_train, c_train)

C:\Users\Sai\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0)

In [28]:
predictions = clf4.predict(b_test)

In [29]:
accuracy=clf4.score(b_test,c_test)

In [30]:
accuracy

0.46003552397868563

In [31]:
# training a DescisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
clf_tree = DecisionTreeClassifier(max_depth = 2).fit(b_train, c_train)
predictions = clf_tree.predict(b_test)

In [32]:
accuracy=clf_tree.score(b_test,c_test)

In [33]:
accuracy

0.60657193605683835

In [34]:
from sklearn.ensemble import GradientBoostingClassifier

In [35]:
clf=GradientBoostingClassifier()

In [36]:
clf.fit(b_train,c_train)

C:\Users\Sai\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [37]:
accuracy=clf.score(b_test,c_test)

In [38]:
accuracy

0.71847246891651861